In [29]:
import duckdb as ddb
import pandas as pd

Establish a connection to the DuckDB database file and

con  # Display the connection object to verify it is active

In [30]:
con = ddb.connect("../air_quality.db")
con

In [31]:
df = con.query("select * from raw.air_quality_data where parameter in ('so2', 'pm10', 'pm25')").to_df()
#len(df)
df.head()

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
0,225405,1303524,Bongani Mabaso Eco Park-237701,2024-02-29 23:00:00,-26.77761,27.837328,pm25,µg/m³,22.095,03,2024,2025-12-23 01:30:44.939410
1,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 00:00:00,-26.77761,27.837328,pm25,µg/m³,23.414,03,2024,2025-12-23 01:30:44.939410
2,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 01:00:00,-26.77761,27.837328,pm25,µg/m³,25.818,03,2024,2025-12-23 01:30:44.939410
3,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 02:00:00,-26.77761,27.837328,pm25,µg/m³,30.793,03,2024,2025-12-23 01:30:44.939410
4,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 03:00:00,-26.77761,27.837328,pm25,µg/m³,36.194,03,2024,2025-12-23 01:30:44.939410


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942 entries, 0 to 941
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   location_id         942 non-null    int64         
 1   sensors_id          942 non-null    int64         
 2   location            942 non-null    object        
 3   datetime            942 non-null    datetime64[us]
 4   lat                 942 non-null    float64       
 5   lon                 942 non-null    float64       
 6   parameter           942 non-null    object        
 7   units               942 non-null    object        
 8   value               942 non-null    float64       
 9   month               942 non-null    object        
 10  year                942 non-null    int64         
 11  ingestion_datetime  942 non-null    datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 88.4+ KB


In [33]:
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,year,ingestion_datetime
count,942.0,9.420000e+02,942,942.00000,9.420000e+02,942.000000,942.0,942
mean,225405.0,1.303565e+06,2024-03-05 16:55:01.910828,-26.77761,2.783733e+01,16.887253,2024.0,2026-01-07 23:49:22.322793
min,225405.0,1.303524e+06,2024-02-29 23:00:00,-26.77761,2.783733e+01,0.000043,2024.0,2025-12-23 01:30:44.939410
25%,225405.0,1.303524e+06,2024-03-02 19:00:00,-26.77761,2.783733e+01,0.007779,2024.0,2025-12-23 01:30:44.939410
50%,225405.0,1.303541e+06,2024-03-05 13:00:00,-26.77761,2.783733e+01,13.184000,2024.0,2026-01-07 23:49:22.322793
75%,225405.0,1.303631e+06,2024-03-07 18:00:00,-26.77761,2.783733e+01,24.038000,2024.0,2026-01-23 22:07:59.706176
max,225405.0,1.303631e+06,2024-03-12 10:00:00,-26.77761,2.783733e+01,152.557000,2024.0,2026-01-23 22:07:59.706176
std,0.0,4.697408e+01,NaN,0.00000,7.109202e-15,19.683701,0.0,NaN


In [34]:
df.describe(include='O')

,location,parameter,units,month
count,942,942,942,942
unique,1,3,2,1
top,Bongani Mabaso Eco Park-237701,pm25,µg/m³,03
freq,942,314,628,942


In [35]:
df[df.duplicated(subset=["location_id","datetime", "parameter", "units", "value", "month", "year", "ingestion_datetime"])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [36]:
df.groupby(by="parameter", as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,month,year,ingestion_datetime
0,pm10,314,314,314,314,314,314,314,314,314,314,314
1,pm25,314,314,314,314,314,314,314,314,314,314,314
2,so2,314,314,314,314,314,314,314,314,314,314,314


In [37]:
df.groupby(by=["parameter", "units"], as_index=False).count()

,parameter,units,location_id,sensors_id,location,datetime,lat,lon,value,month,year,ingestion_datetime
0,pm10,µg/m³,314,314,314,314,314,314,314,314,314,314
1,pm25,µg/m³,314,314,314,314,314,314,314,314,314,314
2,so2,ppm,314,314,314,314,314,314,314,314,314,314


In [38]:
con.close()